In [1]:
import numpy as np
import pandas as pd
import gc
from sklearn import preprocessing
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(font_scale=1.5)

In [2]:
prop = pd.read_csv('../input/properties_2016.csv')
prop.columns.tolist() 

/home/ozkan/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (22,32,34,49,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


['parcelid',
 'airconditioningtypeid',
 'architecturalstyletypeid',
 'basementsqft',
 'bathroomcnt',
 'bedroomcnt',
 'buildingclasstypeid',
 'buildingqualitytypeid',
 'calculatedbathnbr',
 'decktypeid',
 'finishedfloor1squarefeet',
 'calculatedfinishedsquarefeet',
 'finishedsquarefeet12',
 'finishedsquarefeet13',
 'finishedsquarefeet15',
 'finishedsquarefeet50',
 'finishedsquarefeet6',
 'fips',
 'fireplacecnt',
 'fullbathcnt',
 'garagecarcnt',
 'garagetotalsqft',
 'hashottuborspa',
 'heatingorsystemtypeid',
 'latitude',
 'longitude',
 'lotsizesquarefeet',
 'poolcnt',
 'poolsizesum',
 'pooltypeid10',
 'pooltypeid2',
 'pooltypeid7',
 'propertycountylandusecode',
 'propertylandusetypeid',
 'propertyzoningdesc',
 'rawcensustractandblock',
 'regionidcity',
 'regionidcounty',
 'regionidneighborhood',
 'regionidzip',
 'roomcnt',
 'storytypeid',
 'threequarterbathnbr',
 'typeconstructiontypeid',
 'unitcnt',
 'yardbuildingsqft17',
 'yardbuildingsqft26',
 'yearbuilt',
 'numberofstories',
 'firep

In [3]:
geocolumns = [  'latitude', 'longitude','propertycountylandusecode', 
              'propertylandusetypeid', 'propertyzoningdesc','regionidcity',
              'regionidcounty', 'regionidneighborhood', 'regionidzip',
              'censustractandblock', 'rawcensustractandblock']

In [4]:
geoprop = prop[geocolumns]

In [5]:
missingcount = geoprop.isnull().sum(axis=0)
plt.figure( figsize = (16,8) )
plot= sns.barplot( x = geocolumns, y = missingcount.values )
plt.setp( plot.get_xticklabels(), rotation = 45 )
missingcount

latitude                       11437
longitude                      11437
propertycountylandusecode      12277
propertylandusetypeid          11437
propertyzoningdesc           1006588
regionidcity                   62845
regionidcounty                 11437
regionidneighborhood         1828815
regionidzip                    13980
censustractandblock            75126
rawcensustractandblock         11437
dtype: int64

In [6]:
geoprop.dropna( axis = 0, subset = [ 'latitude', 'longitude' ], inplace = True )

/home/ozkan/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [7]:
missingcount = geoprop.isnull().sum(axis=0)
plt.figure( figsize = (16,8) )
plot= sns.barplot( x = geocolumns, y = missingcount.values )
plt.setp( plot.get_xticklabels(), rotation = 45 )
missingcount

latitude                           0
longitude                          0
propertycountylandusecode        840
propertylandusetypeid              0
propertyzoningdesc            995151
regionidcity                   51408
regionidcounty                     0
regionidneighborhood         1817378
regionidzip                     2543
censustractandblock            63689
rawcensustractandblock             0
dtype: int64

In [8]:
geoprop.loc[:,'latitude'] = geoprop.loc[:,'latitude']/1e6
geoprop.loc[:,'longitude'] = geoprop.loc[:,'longitude']/1e6

maxlat = (geoprop['latitude']).max()
maxlon = (geoprop['longitude']).max()
minlat = (geoprop['latitude']).min()
minlon = (geoprop['longitude']).min()
print('maxlat {} minlat {} maxlon {} minlon {}'.format(maxlat, minlat, maxlon, minlon))

CAparms = { 'llcrnrlat' : minlat,
                     'urcrnrlat' : maxlat+0.2,
                     'llcrnrlon' : maxlon-2.5,
                     'urcrnrlon' :minlon+2.5 }

maxlat 34.81965 minlat 33.324388 maxlon -117.554316 minlon -119.47578


/home/ozkan/anaconda2/lib/python2.7/site-packages/pandas/core/indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [10]:
from sklearn import neighbors
from sklearn.preprocessing import OneHotEncoder

In [13]:
def fillna_knn( df, base, target, fraction = 1, threshold = 10 ):
    assert isinstance( base , list ) or isinstance( base , np.ndarray ) and isinstance( target, str ) 
    whole = [ target ] + base
    
    miss = df[target].isnull()
    notmiss = ~miss 
    nummiss = miss.sum()
    
    enc = OneHotEncoder()
    X_target = df.loc[ notmiss, whole ].sample( frac = fraction )
    
    enc.fit( X_target[ target ].unique().reshape( (-1,1) ) )
    
    Y = enc.transform( X_target[ target ].values.reshape((-1,1)) ).toarray()
    X = X_target[ base  ]
    
    print( 'fitting' )
    n_neighbors = 10
    clf = neighbors.KNeighborsClassifier( n_neighbors, weights = 'uniform' )
    clf.fit( X, Y )
    
    print( 'the shape of active features: ' ,enc.active_features_.shape )
    
    print( 'predicting' )
    Z = clf.predict(geoprop.loc[ miss, base  ])
    
    numunperdicted = Z[:,0].sum()
    if numunperdicted / nummiss *100 < threshold :
        print( 'writing result to df' )    
        df.loc[ miss, target ]  = np.dot( Z , enc.active_features_ )
        print( 'num of unperdictable data: ', numunperdicted )
        return enc
    else:
        print( 'out of threshold: {}% > {}%'.format( numunperdicted / nummiss *100 , threshold ) )

In [14]:
fillna_knn( df = geoprop,
                  base = [ 'latitude', 'longitude' ] ,
                  target = 'regionidcity', fraction = 0.15 )

fitting
('the shape of active features: ', (185,))
predicting


ValueError: Found array with 0 sample(s) (shape=(0, 2)) while a minimum of 1 is required.